In [1]:
using PauliPropagation

### X drive to ZZ target

In [11]:
#define hyperparams
nq = 4
del_t = 1.0
P= 20
tau = del_t * P # tau is total annealing time, divided into P steps/layers of circuit
topology = bricklayertopology(nq; periodic=false)
circuit = tfitrottercircuit(nq, P; topology=topology, start_with_ZZ=false) # 1st order trotter
#println(circuit)
nbonds = nq - 1 # if periodic nq bonds, else nq-1 (1d chain)
total = P * (nq + nbonds)
params  = Vector{Float64}(undef, total)

hbar = 1.0
G = -1.0
J = 1.0

idx = 1
for m in 1:P # layers of trotterized circuit
    #s_m = m / P
    s_m = (m - 0.5) / P #evaluate at middle of trotter slice

    gamma_m = s_m * del_t /hbar
    #println(gamma_m)   # paper Eq. (12)
    beta_m = (1-s_m) * del_t / hbar # paper Eq. (13a) - 1st order trotter
    #println(beta_m)
    theta_X = 2*beta_m * G 
    ##println(theta_X)              # angle for e^{-i theta_X X_j/2} - PauliRotation implementation in PP
    theta_ZZ = 2*gamma_m * J              # angle for e^{-i theta_ZZ Z_j Z_{j+1}/2}
    ##println(theta_ZZ)
    @inbounds fill!(view(params, idx:idx+nq-1), theta_X);             idx += nq # set X angles,then shift the index
    #println(params)
    @inbounds fill!(view(params, idx:idx+nbonds-1), theta_ZZ);       idx += nbonds
    #println(params)
end
H_ZZ = PauliSum(nq)
for pair in topology
    add!(H_ZZ, [:Z, :Z], collect(pair), J)
end

max_weight = 10
min_abs_coeff = 1e-7
propagated = propagate(circuit, H_ZZ, params)#; max_weight, min_abs_coeff)
println(overlapwithplus(propagated))


-2.9958176743671237


This works well, also when scaling up. Now, we want to do DQA for more complex Hamiltonians.

### X drive to TFIM target

In [12]:
#define hyperparams
using PauliPropagation  
nq = 4
del_t = 0.1
P= 500
tau = del_t * P # tau is total annealing time, divided into P steps/layers of circuit
topology = bricklayertopology(nq; periodic=false)
circuit = tfitrottercircuit(nq, P; topology=topology, start_with_ZZ=false) # 1st order trotter
#println(circuit)
nbonds = nq - 1 # if periodic nq bonds, else nq-1 (1d chain)
total = P * (nq + nbonds)
params  = Vector{Float64}(undef, total)

hbar = 1.0
G = 1.0
J = 1.0

idx = 1
for m in 1:P # layers of trotterized circuit
    # s_m = m / P
    s_m = (m - 0.5) / P

    gamma_m = J* s_m * del_t /hbar
    #println(gamma_m)   # paper Eq. (12)
    beta_m = G*del_t*(2*s_m-1)/ hbar #(1-s_m) * del_t / hbar # paper Eq. (13a) - 1st order trotter
    #println(beta_m)
    theta_X = 2*beta_m
    ##println(theta_X)              # angle for e^{-i theta_X X_j/2} - PauliRotation implementation in PP
    theta_ZZ = 2*gamma_m              # angle for e^{-i theta_ZZ Z_j Z_{j+1}/2}, J=1
    ##println(theta_ZZ)
    @inbounds fill!(view(params, idx:idx+nq-1), theta_X);             idx += nq # set X angles,then shift the index
    #println(params)
    @inbounds fill!(view(params, idx:idx+nbonds-1), theta_ZZ);       idx += nbonds
    #println(params)
end
H_tfim = PauliSum(nq)

for qind in 1:nq
    add!(H_tfim, :X, qind, G)
end

for pair in topology
    add!(H_tfim, [:Z, :Z], collect(pair), J)
end


max_weight = 10
min_abs_coeff = 1e-7
propagated = propagate(circuit, H_tfim, params)#; max_weight, min_abs_coeff)
println(overlapwithplus(propagated))

-4.670877390760635


This also works! We can make it more precise by increasing P, but we converge very slow.
Let's try the same for the Heisenberg Hamiltonian:

### X drive to Heisenberg Hamiltonian
- Watch out: Heisenberg Hamiltonian commutes with the driver (X) that we used above
- If we want to keep the plus state as the departure state for the interpolating Hamiltonian, we need to break the SU(2) symmetry of the Heisenberg Hamiltonian
- One way to do this is to adjust the coeffs of the driver to all be positive but slightly different

In [11]:
using PauliPropagation

# --- hyperparams ---
nq  = 3
topology = bricklayertopology(nq; periodic=false)
nbonds = nq - 1

del_t = 0.1
P= 100000
tau = del_t * P

hbar = 1.0
G = 1.0
J = 1.0
g = [1,1.6,0.4] # break SU(2) symmetry of Heisenberg model
# --- circuit: [X]*nq → [XX]*nbonds → [YY]*nbonds → [ZZ]*nbonds per layer ---
function dqa_heisenbergtrottercircuit(nqubits::Int, nlayers::Int; topology)
    circuit = Gate[]
    for _ in 1:nlayers
        for q in 1:nqubits
            push!(circuit, PauliRotation(:X, q))
        end
        rxxlayer!(circuit, topology)
        ryylayer!(circuit, topology)
        rzzlayer!(circuit, topology)
    end
    return circuit
end

circuit = dqa_heisenbergtrottercircuit(nq, P; topology)

# --- params packed as [X | XX | YY | ZZ] for each layer ---
total  = P*(nq + 3*nbonds)
params = Vector{Float64}(undef, total)
println("Total params = ", total)
idx = 1
for m in 1:P
    s_m = (m - 0.5) / P
    beta   = -g .*[(1- s_m) * del_t / hbar]     # X coefficient
    gamma  =  J * s_m       * del_t / hbar     # XX/YY/ZZ coefficient

    θX  = 2 .*beta
    θXX = 2*gamma
    θYY = 2*gamma
    θZZ = 2*gamma
    #println("done")
    #@inbounds fill!(view(params, idx:idx+nq-1), θX);                  idx += nq
    for i in 1:nq
        @inbounds fill!(view(params, idx-1+i), θX[i]) # to avoid broadcasting
    end
    #println(params)
    idx += nq
    @inbounds fill!(view(params, idx:idx+nbonds-1), θXX);             idx += nbonds
    #println(params)
    @inbounds fill!(view(params, idx:idx+nbonds-1), θYY);             idx += nbonds
    #println(params)
    @inbounds fill!(view(params, idx:idx+nbonds-1), θZZ);             idx += nbonds
    #println(params)
end

# --- final observable: Heisenberg only (no X term) ---
H_heis = PauliSum(nq)
for (i,j) in topology
    add!(H_heis, [:X,:X], [i,j], J)
    add!(H_heis, [:Y,:Y], [i,j], J)
    add!(H_heis, [:Z,:Z], [i,j], J)
end
max_weight = 5
min_abs_coeff = 1e-7
propH = propagate(circuit, H_heis, params; max_weight, min_abs_coeff)

# Heisenberg-picture readout: ⟨+| U† H_heis U |+⟩
println("E_final = ", overlapwithplus(propH))


Total params = 900000
E_final = -3.8264365959710824


This seems to work, but is far to comp expensive. We need a different approach to DQA for the Heisenberg Hamiltonian / a different way to pre-propagate the state.